### Download data 

In [ ]:
%%bash 
cd SageMaker 
wget -O data.zip https://tinyurl.com/y8dty9qu
unzip data.zip

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket() 
!aws s3 cp --recursive resnet-aoi-2 s3://{bucket}/resnet-aoi-2 

In [ ]:
input_data = sagemaker.inputs.TrainingInput("s3://{}/resnet-aoi-2/".format(bucket))
data_channel = {'training':input_data}
print(input_data)

### Let's switch to Sagemaker Studio to use JumpStart! 

### Let's switch back to this Notebook:) 

In [ ]:
hyperparameters = {
    "adam-learning-rate":"0.05",
    "batch-size":"4",
    "epochs":"3", 
    "model-artifact-bucket":"jumpstart-cache-prod-us-west-2", 
    "model-artifact-key":"tensorflow-training/train-tensorflow-ic-resnet-50-classification-1.tar.gz"
}

In [ ]:
from sagemaker.tensorflow import TensorFlow



estimator = TensorFlow(
                        base_job_name='resnet-aoi',
                        source_dir='code',
                        entry_point='transfer_learning_single.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.3.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=1,
                        # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        hyperparameters = hyperparameters, 
                        sagemaker_session=sagemaker_session,
                        disable_profiler=True,
                        debugger_hook_config=False,
                        # Training using SMDataParallel Distributed Training Framework
#                         distribution={'smdistributed':{
#                                             'dataparallel':{
#                                                     'enabled': True
#                                              }
#                                       }}
                        )

In [ ]:
estimator.fit(inputs=data_channel)


In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data